# Análisis Exploratorio de Datos – Fuentes UNID

En este notebook se realiza un análisis exploratorio de las fuentes no identificadas (UNIDs) contenidas en el archivo `unids_3F_beta_err_names.txt`. Estas fuentes no cuentan con una etiqueta de clase conocida (a diferencia de los datos ASTRO o DM simulados), pero se espera que puedan incluir candidatas a materia oscura.

Cada instancia incluye las siguientes variables:

- **E_peak**: Energía pico (sin logaritmar).
- **beta**: Curvatura espectral.
- **sigma_det**: Significancia de detección.
- **beta_Rel**: Error relativo sobre la curvatura.
- **number**: Identificador único de cada fuente.

Este análisis busca comprender la distribución de estas variables, detectar patrones, posibles agrupaciones o comportamientos atípicos que puedan indicar candidatos interesantes para detección de anomalías con modelos no supervisados como One-Class SVM.

In [ ]:
# Transformación Logarítmica y Verificación de Compatibilidad - UNIDs Fermi-LAT
# TFG: Utilización de técnicas de ML a datos del satélite Fermi-Lat para detección de posibles fuentes de materia oscura
# Objetivo: Transformar datos UNIDs a escala logarítmica para compatibilidad con modelo OneClassSVM

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 10)
plt.rcParams['font.size'] = 12

In [ ]:
# Definir nombres de las columnas para UNIDs
unids_features = ['E_peak', 'beta', 'sigma_det', 'beta_Rel', 'number']

# Cargar los datos UNIDs
unids_df = pd.read_csv('../../data/raw/unids_3F_beta_err_names.txt', 
                       sep="\s+", names=unids_features, engine='python', skiprows=1)

print(f"Dimensiones del dataset UNIDs: {unids_df.shape}")
print(f"Número total de fuentes no identificadas: {len(unids_df)}")

In [ ]:
# Información básica del dataset
print("\nInformación del dataset UNIDs:")
unids_df.info()

In [ ]:
unids_df.describe()

In [ ]:
print(f"\nPrimeras 10 fuentes UNIDs:")
unids_df.head(10)

In [ ]:
print(f"\nÚltimas 5 fuentes UNIDs:")
unids_df.tail()

In [ ]:
# Verificar el rango de IDs
print(f"\nRango de IDs de fuentes UNIDs:")
print(f"ID mínimo: {unids_df['number'].min()}")
print(f"ID máximo: {unids_df['number'].max()}")
print(f"IDs únicos: {unids_df['number'].nunique()}")

if unids_df['number'].nunique() == len(unids_df):
    print("Cada fuente UNID tiene un ID único")
else:
    print("Hay IDs duplicados en el dataset")

In [ ]:
# Variables de características (excluyendo el ID)
unids_feature_cols = ['E_peak', 'beta', 'sigma_det', 'beta_Rel']

print("Estadísticas descriptivas de fuentes UNIDs:")
unids_stats = unids_df[unids_feature_cols].describe()
print(unids_stats)

# Verificar valores faltantes
print(f"\nVerificación de valores faltantes:")
missing_values = unids_df.isnull().sum()
print(missing_values)

if missing_values.sum() == 0:
    print("No hay valores faltantes en el dataset UNIDs")
else:
    print("Se encontraron valores faltantes")

# Verificar valores duplicados (excluyendo el ID)
duplicates = unids_df[unids_feature_cols].duplicated().sum()
print(f"\nFilas duplicadas (sin considerar ID): {duplicates}")

## Histogramas de cada variable (feature)

In [ ]:
# Histogramas de todas las variables UNIDs
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.ravel()

for i, col in enumerate(unids_feature_cols):
    # Histograma
    axes[i].hist(unids_df[col], bins=40, alpha=0.7, color='lightgreen', edgecolor='black')
    axes[i].set_title(f'Distribución de {col} - Fuentes UNIDs')
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('Frecuencia')
    axes[i].grid(True, alpha=0.3)
    
    # Añadir estadísticas básicas
    mean_val = unids_df[col].mean()
    std_val = unids_df[col].std()
    median_val = unids_df[col].median()
    
    axes[i].axvline(mean_val, color='red', linestyle='--', alpha=0.8, 
                   label=f'Media: {mean_val:.3f}')
    axes[i].axvline(median_val, color='blue', linestyle='-.', alpha=0.8, 
                   label=f'Mediana: {median_val:.3f}')
    axes[i].axvline(mean_val + std_val, color='orange', linestyle=':', alpha=0.8, 
                   label=f'+1σ: {mean_val + std_val:.3f}')
    axes[i].axvline(mean_val - std_val, color='orange', linestyle=':', alpha=0.8, 
                   label=f'-1σ: {mean_val - std_val:.3f}')
    axes[i].legend(fontsize=10)

plt.suptitle('Distribuciones de Variables - Fuentes UNIDs', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Matriz de correlación para UNIDs
unids_correlation = unids_df[unids_feature_cols].corr()
print("Matriz de correlación entre variables UNIDs:")
print(unids_correlation)

# Visualización de la matriz de correlación
plt.figure(figsize=(10, 8))
mask = np.triu(np.ones_like(unids_correlation, dtype=bool))
sns.heatmap(unids_correlation, annot=True, cmap='RdBu_r', center=0, 
            square=True, mask=mask, cbar_kws={"shrink": .8})
plt.title('Matriz de Correlación - Fuentes UNIDs')
plt.tight_layout()
plt.show()

In [ ]:
# Identificar correlaciones significativas
print("\nCorrelaciones significativas en UNIDs (|r| > 0.3):")
for i in range(len(unids_correlation.columns)):
    for j in range(i+1, len(unids_correlation.columns)):
        corr_val = unids_correlation.iloc[i, j]
        if abs(corr_val) > 0.3:
            print(f"{unids_correlation.columns[i]} - {unids_correlation.columns[j]}: {corr_val:.3f}")

In [ ]:
# Crear scatter plots para pares de variables
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

# Pares de variables importantes
pairs = [
    ('E_peak', 'beta'),
    ('E_peak', 'sigma_det'),
    ('beta', 'sigma_det'),
    ('beta', 'beta_Rel'),
    ('sigma_det', 'beta_Rel'),
    ('E_peak', 'beta_Rel')
]

for i, (x_var, y_var) in enumerate(pairs):
    # Scatter plot
    scatter = axes[i].scatter(unids_df[x_var], unids_df[y_var], 
                             alpha=0.6, c='darkgreen', s=30)
    
    axes[i].set_xlabel(x_var)
    axes[i].set_ylabel(y_var)
    axes[i].set_title(f'{x_var} vs {y_var} - Fuentes UNIDs')
    axes[i].grid(True, alpha=0.3)
    
    # Añadir línea de tendencia si hay correlación significativa
    if abs(unids_correlation.loc[x_var, y_var]) > 0.3:
        z = np.polyfit(unids_df[x_var], unids_df[y_var], 1)
        p = np.poly1d(z)
        axes[i].plot(unids_df[x_var], p(unids_df[x_var]), "r--", alpha=0.8,
                    label=f'r = {unids_correlation.loc[x_var, y_var]:.3f}')
        axes[i].legend()

plt.suptitle('Análisis Bivariado - Fuentes UNIDs', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()


In [ ]:
"""# Convertir las columnas a logaritmo de base 10
unids_df['E_peak'] = np.log10(unids_df['E_peak'])
unids_df['beta'] = np.log10(unids_df['beta'])
unids_df['sigma_det'] = np.log10(unids_df['sigma_det'])
unids_df['beta_Rel'] = np.log10(unids_df['beta_Rel'])
# Ver muestra
unids_df.head()"""

## Observaciones del Análisis Exploratorio (EDA) – Fuentes UNID

A continuación se detallan los principales hallazgos obtenidos del análisis de las variables contenidas en las fuentes no identificadas (UNIDs):

### Distribuciones de variables
- **E_peak** presenta una fuerte asimetría positiva (cola larga), con la mayoría de valores muy concentrados por debajo de 100, pero con valores extremos que superan los 1000. Esto sugiere la presencia de posibles outliers o escalas diferentes de emisión en algunas fuentes.
- **beta** muestra una distribución sesgada hacia valores bajos, con una concentración destacada entre 0.0 y 0.4. Este patrón puede estar asociado a formas espectrales más típicas, mientras que valores mayores podrían representar fuentes más inusuales.
- **beta_Rel** y **sigma_det** también muestran colas largas, lo que indica variabilidad en la calidad o fiabilidad de las mediciones espectrales.
- La transformación logarítmica de estas variables ayuda a **reducir la asimetría y comprimir outliers**, como se evidencia en la última figura (`log10(E_peak)` vs `log10(beta)`).

### Correlación entre variables
- El mapa de correlación muestra **correlaciones débiles** entre las variables (`|r| < 0.25`), lo que sugiere que estas características aportan información relativamente independiente.
- La variable `beta_Rel` está moderadamente correlacionada negativamente con `beta`, lo que podría indicar que la curvatura espectral más baja tiende a estar mejor caracterizada.

### Relaciones bivariadas
- El scatter plot `E_peak` vs `beta` evidencia **grandes concentraciones en regiones específicas del espacio**, pero también algunos puntos claramente aislados (posibles candidatos a anomalía).
- Al aplicar la escala logarítmica (`log10`), se observa un **grupo denso bien delimitado** junto con varios puntos alejados, lo que refuerza la necesidad de utilizar un modelo de detección de anomalías.

---

## Conclusión

Los datos de las fuentes UNID presentan una alta variabilidad en sus características espectrales, así como valores extremos que podrían indicar fenómenos no convencionales. Esta exploración apoya el uso de modelos de aprendizaje no supervisado, como **One-Class SVM**, para identificar observaciones que se desvíen significativamente del comportamiento de fuentes astrofísicas conocidas.
